# EXERCÍCIO 2

E se, além da sena, desejo o resultado da quina e da quadra. O ideal é uma função ampliada que retorne, para cada tipo de prêmio, a quantidade de ganhadores e o valor do prêmio dividido. Antes de desenvolver a função, esboce em sites de formatação JSON como seria este dicionário, o qual será montado dentro de sua função. Aqui, métodos de string como startswith, strip e split serão úteis.

In [5]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup as soup

# Pegar número do concurso
numConcurso = input("Número do concurso: ")

# Fazer a raspagem dos dados
url = 'https://www.loteriaseresultados.com.br/index.php/megasena/concurso/'+numConcurso
driver = webdriver.Firefox()
driver.get(url)

# tempo para página carregar
time.sleep(15)

# Pegando valores das dezenas sorteadas
content = driver.find_elements_by_css_selector("span[class='white--text font-weight-bold']")
valores = []
for n in range(6):
    valores.insert(n, content[n].get_attribute('innerHTML'))

# Pegar os ganhadores
content2 = driver.find_elements_by_css_selector("td[class='text-center']")
ganhadores = []
for n in range(6):
    ganhadores.insert(n, content2[n].get_attribute('innerHTML'))

#Pegar valor ganho 
content3 = driver.find_elements_by_css_selector("td[class='text-right']")
dinheiro = []
for n in range(3):
    dinheiro.insert(n, content3[n].get_attribute('innerHTML'))

driver.close()

qtd = []
for i in range(6):
    if(i%2):
        if(ganhadores[i] == "Não houve acertador!"):
            qtd.append('0')
        else:
            qtd.append(ganhadores[i])

#Armazenar retorno da função no mongodb
from pymongo import MongoClient
localConn = MongoClient('mongodb://127.0.0.1:27017')

db = localConn.concursos 
tb = db.megasena 

#Organizar os dados
for j in range(3): qtd[j] = qtd[j].replace(' ganhadores', '')

for d in range(3):
    if('-' in dinheiro[d]):
        dinheiro[d] = '0'
    else:
        dinheiro[d] = dinheiro[d].replace('R$ ', '')
        x = dinheiro[d].split(',')
        dinheiro[d] = x[0]

for i in range(6): valores[i] = int(valores[i])

data = {
    'Concurso': numConcurso, 
    'Dezenas Sorteadas': valores,
    'Sena': {
        'Quantidade de ganhadores': round(float(qtd[0])),
        'Valor do prêmio dividido': float(dinheiro[0])
    },
    'Quina': {
        'Quantidade de ganhadores': round(float(qtd[1])),
        'Valor do prêmio dividido': float(dinheiro[1])
    },
    'Quadra': {
        'Quantidade de ganhadores': round(float(qtd[2])),
        'Valor do prêmio dividido': float(dinheiro[2])
    }
}

print(data)

tb.insert_one(data)
print('\nDocumento inserido com sucesso...')
localConn.close()

Número do concurso: 2340
{'Concurso': '2340', 'Dezenas Sorteadas': [16, 21, 28, 41, 49, 51], 'Sena': {'Quantidade de ganhadores': 0, 'Valor do prêmio dividido': 0.0}, 'Quina': {'Quantidade de ganhadores': 24, 'Valor do prêmio dividido': 93.309}, 'Quadra': {'Quantidade de ganhadores': 2, 'Valor do prêmio dividido': 1.328}}

Documento inserido com sucesso...
